In [5]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import SparkSession
from dotenv import load_dotenv
from datetime import datetime
import yfinance as yf
import pandas as pd
import os

load_dotenv(override=True)

postgres_v = os.getenv("POSTGRES_VERSION")
postgres_url = os.getenv("POSTGRES_URL")
postgres_user = os.getenv("POSTGRES_USER")
postgres_pass = os.getenv("POSTGRES_PASSWORD")
postgres_table = os.getenv("POSTGRES_TABLE")
format_file = os.getenv("FORMAT_FILE")
_mode = os.getenv("MODE")

config_ = postgres_v

column_1_name = os.getenv("COLUMN_1")
column_2_name = os.getenv("COLUMN_2")
column_3_name = os.getenv("COLUMN_3")
column_4_name = os.getenv("COLUMN_4")
column_5_name = os.getenv("COLUMN_5")
column_6_name = os.getenv("COLUMN_6")
column_7_name = os.getenv("COLUMN_7")
column_8_name = os.getenv("COLUMN_8")

list_remove = ['GEV','SOLV','VLTO','BF.B','BRK.B']

class ListSAndP500:
    def __init__(self):
        """
        Initialize the ListSAndP500 class

        Attributes:
        tickers_string (list): A list of stock symbols in string format
        tickers_list (list): A list of stock symbols in list format

        """
        _tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
        _tickers = _tickers.Symbol.to_list()
        self.tickers_string = [i.replace('.','-') for i in _tickers]
        _tickers_list_transform_ = [i if i not in list_remove else False for i in _tickers]
        self.tickers_list = [i for i in _tickers_list_transform_ if i]
        
list_of_symbols__ = ListSAndP500().tickers_list

class YahooFinance:
    def __init__(self, list_of_symbols, start, end):
        self.schema = StructType([
            StructField(column_1_name, StringType(), True),
            StructField(column_2_name, StringType(), True),
            StructField(column_3_name, StringType(), True),
            StructField(column_4_name, StringType(), True),
            StructField(column_5_name, StringType(), True),
            StructField(column_6_name, StringType(), True),
            StructField(column_7_name, StringType(), True),
            StructField(column_8_name, StringType(), True)
        ])

        self.symbols = list_of_symbols
        self.interval = '1d'
        self.start = start
        self.end = end
        self.results = self.process_data()

    def process_data(self):
        """
        Process the historical stock data for the stock symbols
        """
        data = self.get_data()
        return self.transform_data(data)

    def get_data(self):
        """
        Get historical stock data from Yahoo Finance API using yfinance library

        Returns:
        DataFrame: A DataFrame containing historical stock data
        """
        try:
            data = yf.download(
                self.symbols,
                start=self.start,
                end=self.end,
                interval=self.interval,
                ignore_tz=True,
                threads=5,
                timeout=60,
                progress=True
            )
            return data
        except Exception as e:
            print(f"Error downloading data: {e}")
            return None

    def transform_data(self, df):
        """
        Transform the historical stock data into a format that can be stored in a database FactPrices table

        Args:
        df (DataFrame): A DataFrame containing historical stock data

        Returns:
        DataFrame: A DataFrame containing transformed historical stock data with the following columns:
        - stock_id (str): The stock symbol
        - date (str): The date of the stock data
        - open (float): The opening price of the stock
        - high (float): The highest price of the stock
        - low (float): The lowest price of the stock
        - close (float): The closing price of the stock
        - volume (int): The volume of the stock
        - adjusted_close (float): The adjusted closing price of the stock

        """
        # Reset the index to turn the MultiIndex into columns
        df = df.reset_index()

        # Create a list to store transformed records
        records = []

        # Iterate over each row and stock symbol
        for index, row in df.iterrows():
            date = row[('Date', '')]
            for stock in self.symbols:
                try:
                    record = {
                        column_1_name: stock,
                        column_2_name: date,
                        column_3_name: row[('Open', stock)],
                        column_4_name: row[('High', stock)],
                        column_5_name: row[('Low', stock)],
                        column_6_name: row[('Close', stock)],
                        column_7_name: row[('Volume', stock)],
                        column_8_name: row[('Adj Close', stock)]
                    }
                    records.append(record)
                except KeyError as e:
                    print(f"KeyError: {e} for stock: {stock} on date: {date}")

        # Convert the list of records into a DataFrame
        return pd.DataFrame(records)

In [6]:
transformed_data = YahooFinance(list_of_symbols__, '2015-01-01', datetime.now().strftime('%Y-%m-%d')).results
print(transformed_data.head())

[*********************100%%**********************]  498 of 498 completed


  stock_id       date        open        high         low       close  \
0      MMM 2015-01-02  137.717392  138.026749  136.061874  137.173920   
1      AOS 2015-01-02   28.309999   28.415001   27.775000   28.004999   
2      ABT 2015-01-02   45.250000   45.450001   44.639999   44.900002   
3     ABBV 2015-01-02   65.440002   66.400002   65.440002   65.889999   
4      ACN 2015-01-02   89.669998   90.089996   88.430000   88.839996   

      volume  adjusted_close  
0  2531214.0       92.933624  
1  1540200.0       24.154337  
2  3216600.0       37.481434  
3  5086100.0       44.314434  
4  2021300.0       75.950920  


In [7]:
len(transformed_data)

1184742

In [49]:
filtered_data = transformed_data[transformed_data['stock_id'] == list_of_symbols__[0]]
filtered_data

,stock_id,date,open,high,low,close,volume,adjusted_close
0,MMM,2015-01-02,137.717392,138.026749,136.061874,137.173920,2531214.0,92.933624
498,MMM,2015-01-05,136.287628,136.822739,133.846161,134.080261,4416708.0,90.837677
996,MMM,2015-01-06,134.464890,134.924744,131.889633,132.650497,4224272.0,89.869041
1494,MMM,2015-01-07,133.695648,134.013382,132.892975,133.612045,3685235.0,90.520477
1992,MMM,2015-01-08,134.322739,136.864548,134.214050,136.814377,3758908.0,92.689987
...,...,...,...,...,...,...,...,...
1182252,MMM,2024-06-10,100.430000,101.839996,100.279999,100.660004,3507600.0,100.660004
1182750,MMM,2024-06-11,100.099998,101.470001,99.230003,101.320000,2831500.0,101.320000
1183248,MMM,2024-06-12,102.620003,103.680000,100.760002,101.209999,3704600.0,101.209999
1183746,MMM,2024-06-13,102.019997,102.800003,100.739998,101.540001,5034700.0,101.540001


In [40]:
unique_dates = pd.DatetimeIndex(transformed_data['date'].unique())

In [46]:
complete_dates = pd.date_range(start='2015-01-01', end=datetime.now().strftime('%Y-%m-%d'))

missing_dates = complete_dates.difference(unique_dates)

# Convert missing dates to a DataFrame for display
missing_dates_df = pd.DataFrame(missing_dates, columns=['missing_dates'])

print(len(missing_dates_df))
print(missing_dates_df.head(20))
print(len(complete_dates))

1075
   missing_dates
0     2015-01-01
1     2015-01-03
2     2015-01-04
3     2015-01-10
4     2015-01-11
5     2015-01-17
6     2015-01-18
7     2015-01-19
8     2015-01-24
9     2015-01-25
10    2015-01-31
11    2015-02-01
12    2015-02-07
13    2015-02-08
14    2015-02-14
15    2015-02-15
16    2015-02-16
17    2015-02-21
18    2015-02-22
19    2015-02-28
3454


In [47]:
data = yf.download(list_of_symbols__, start="2015-01-01", end="2015-01-01")
print(data.head())

[*                      2%%                      ]  9 of 498 completed

$TYL: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$NFLX: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$PODD: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$SLB: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$EMN: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$PH: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AJG: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AVY: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$ALL: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$NEE: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)


[***                    7%%                      ]  33 of 498 completed

$KEYS: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AMT: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$CBRE: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AVGO: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$GPN: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$DHI: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$ES: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$VRSK: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$COP: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$INCY: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$STT: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$LEN: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$ADM: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$REG: po

[*****                 10%%                      ]  48 of 498 completed

...

$DVN: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$BR: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$MDT: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$PARA: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AEE: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$HBAN: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$MTB: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)


[**********************99%%**********************]  494 of 498 completed

$ZBRA: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$LDOS: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AMP: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$COST: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AMCR: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$UNH: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$APA: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$BK: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AZO: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$CPB: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$PM: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$A: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$XYL: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$AAPL: possi

[*********************100%%**********************]  498 of 498 completed

498 Failed downloads:
['TYL', 'NFLX', 'PODD', 'SLB', 'EMN', 'PH', 'AJG', 'AVY', 'ALL', 'NEE', 'KEYS', 'AMT', 'CBRE', 'AVGO', 'GPN', 'DHI', 'ES', 'VRSK', 'COP', 'INCY', 'STT', 'LEN', 'ADM', 'REG', 'BEN', 'HRL', 'FAST', 'NRG', 'TGT', 'CHD', 'MHK', 'KIM', 'NVR', 'CRL', 'CMS', 'LOW', 'MMM', 'IT', 'SJM', 'DLR', 'WEC', 'BALL', 'ECL', 'SPGI', 'SYK', 'APD', 'SNA', 'CHTR', 'TSCO', 'VLO', 'ADI', 'SMCI', 'TPR', 'WY', 'PHM', 'KLAC', 'BWA', 'ISRG', 'MKTX', 'ABBV', 'TMO', 'MCHP', 'SWK', 'DD', 'DVA', 'IEX', 'SBUX', 'HIG', 'MPC', 'APH', 'DE', 'JCI', 'KO', 'LH', 'BAX', 'MAS', 'CDNS', 'MET', 'CSGP', 'CMI', 'MCK', 'BKR', 'NOW', 'RCL', 'AIZ', 'PAYC', 'ILMN', 'HAS', 'AFL', 'ROP', 'MKC', 'MGM', 'IFF', 'AES', 'RL', 'IBM', 'TXN', 'BX', 'BDX', 'SRE', 'GOOGL', 'PANW', 'WBD', 'VTR', 'MA', 'AKAM', 'WST', 'KDP', 'DUK', 'BG', 'LLY', 'WFC', 'ORLY', 'LVS', 'GOOG', 'HLT', 'HCA', 'ELV', 'PG', 'MCO', 'GIS', 'DG', 'IP', 'WYNN', 'NOC', 'EVRG', 'INTU

$MS: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
$MSI: possibly delisted; No price data found  (1d 2015-01-01 -> 2015-01-01)
Empty DataFrame
Columns: [(Adj Close, A), (Adj Close, AAL), (Adj Close, AAPL), (Adj Close, ABBV), (Adj Close, ABNB), (Adj Close, ABT), (Adj Close, ACGL), (Adj Close, ACN), (Adj Close, ADBE), (Adj Close, ADI), (Adj Close, ADM), (Adj Close, ADP), (Adj Close, ADSK), (Adj Close, AEE), (Adj Close, AEP), (Adj Close, AES), (Adj Close, AFL), (Adj Close, AIG), (Adj Close, AIZ), (Adj Close, AJG), (Adj Close, AKAM), (Adj Close, ALB), (Adj Close, ALGN), (Adj Close, ALL), (Adj Close, ALLE), (Adj Close, AMAT), (Adj Close, AMCR), (Adj Close, AMD), (Adj Close, AME), (Adj Close, AMGN), (Adj Close, AMP), (Adj Close, AMT), (Adj Close, AMZN), (Adj Close, ANET), (Adj Close, ANSS), (Adj Close, AON), (Adj Close, AOS), (Adj Close, APA), (Adj Close, APD), (Adj Close, APH), (Adj Close, APTV), (Adj Close, ARE), (Adj Close, ATO), (Adj Close, AVB), (Adj Close, AVGO)